# 2021 Dixie Fire Raster Analysis

The 2021 Dixie Fire began on July 13, 2021 when a large fir tree fell on top of a 12-kilovolt power distribution line operated by Pacific Gas and Electric Company (PG&E), resulting in the ignition of fuels on the ground beneath the transmission lines. By August 6, it has become the largest single wildfire in California's history, primarily impacting Plumas County, CA. Contributing factors to its spread included intensification of drought, overcrowded forest, and poor road access to the initial outbreak area making it difficult for emergency personnel to control the fire.

On September 9, the weather conditions became more favorable with calmer winds, cool overnight temperatures, rain and increased humidity. By September 22, containment reached 95% and on October 1, the last remaining area became devoid of heat according to infrared data.

In [1]:
# **Table of Contents**

#Dixie Fire Analysis Introduction
#Import libraries
#Getting Data
#Exploratory Data Analysis
#Visual Assessment
#Quantitative Assessment
#Image Segmentation and Classification
#Conclusion

# Import libraries

In [ ]:
from arcgis.gis import GIS
from IPython.display import display
gis = GIS()

In [ ]:
import arcgis
from arcgis.raster.functions import *
from ipywidgets import *
import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib import rcParams

# Getting Data

I used USGS Earth explorer's data base to find data. https://earthexplorer.usgs.gov/

I found Landsat 8 Imagery taken from before, during, and after the fire. Pre-fire imagery scene ID: LC08_L1TP_044032_20200622_20200823_02_T1, acquired 06/22/2020. Post-fire imagery scene ID: LC08_L1TP_044032_20211015_20211025_02_T1, acquired 10/15/2021. I then imported the rasters from each scene into ArcGIS Pro as seven single-band TIF files for analysis.

# Exploratory Data Analysis

According to Esri, a natural color orthoimage is a multiband image in which there are three bands, each representing either red, green, or blue light. https://pro.arcgis.com/en/pro-app/latest/help/data/imagery/raster-bands-pro-.htm According to GISGeography.com, band 2 shows visible green, band 3 visible blue, and band 4 visible red. https://gisgeography.com/landsat-8-bands-combinations/
To test the data, I used the geoprocessing tool 'Composite Bands' to form a multiband raster of each data set.

In [ ]:
multiband_raster = 'Data/sat_image.png'
# reading png image file
read = img.imread(multiband_raster)
  
# show image
plt.imshow(read)

This image didn't look very "natural" to me so I created another multiband raster using the bands that USGS recommends
to form a "Reflective Full Resolution Browse" image (Bands 6,5,4)
https://www.usgs.gov/landsat-missions/landsat-collection-2-full-resolution-browse-images

In [ ]:
refl = 'Data/reflective_full_resolution.png'
read = img.imread(refl)
plt.imshow(read)
print('Before Dixie Fire')

I thought that looked a lot better. So I applied the same process to the post-fire data, to see if there were any immediately visible contrasts between the two images, and created a Web Mapping Application with a swipe tool to visualize the differences.

In [ ]:
refl_full_res = gis.content.get('404db775dfc846988c6b02922d4162e8')
print('Visualization of affected areas:')
refl_full_res

# Visual Assessment

The burn scars were very apparent. However, to create an infrared (burn severity) map that highlights visibility of fire and burn scars, the bands 5,3,2 must be combined using composite bands. Healthy vegetation is shown in bright red, while 
stressed vegetation is displayed as dull red.

In [ ]:
%matplotlib inline

# figure size in inches optional
rcParams['figure.figsize'] = 15 ,12

# read images
img_A = img.imread('Data/burn_scars.png')
img_B = img.imread('Data/infrared.png')

# display images
fig, ax = plt.subplots(1,2)
ax[0].imshow(img_B)
ax[1].imshow(img_A)
print('Before and after the Dixie fire')

# Burnt Area Calculation

In order to perform a true analysis of how much vegetation was affected by the fire, I'll need to categorize the
raster image into classes. Using image segmentation and classification we can determine the total amount of vegetation
affected. I performed this through the Image Classification wizard on ArcGIS Pro using the post-fire infrared layer.

I ran an unsupervised classification with a maximum number of 9 classes. I then reclassified those into four categories:
Burnt area, vegetation, water, and barren land. Since the classification tool is not perfect, I had to manually re-
classify some areas, totaling 14 edits.

In [ ]:
%matplotlib inline

rcParams['figure.figsize'] = 15 ,12

img_A = img.imread('Data/burn_scars.png')
img_B = img.imread('Data/reclassified_land_types.png')

fig, ax = plt.subplots(1,2)
ax[0].imshow(img_A)
ax[1].imshow(img_B)
print('Before and after image classification')

In [ ]:
reclassified = gis.content.get('ceb3a6ce0d5f4872ae15b9731c06434a')
print('Comparison of the two layers in a web map:')
reclassified

To calculate the total burnt area in the image, I converted the reclassified layer into polygons using the raster to polygons geoprocessing tool. Then I added a field in the vector layer's attribute table and selected the polygons by attributes with the expression "gridcode is equal to 3" to select those polygons indicating burnt area. In the attribute table I calculated the geometry of the new field I created and selected "area" as the property and "square meters" as the map unit, showing the values for only the burnt area polygons.

In [ ]:
calc = 'Data/calculate_geometry_m2.png'
showcalc = img.imread(calc)
plt.imshow(showcalc)
print('Area calculation of "Burnt area" polygons')

The sum total was 3,763,176,718.

In [ ]:
rcParams['figure.figsize'] = 9 ,6
stats = 'Data/stats2.png'
showstats = img.imread(stats)
plt.imshow(showstats)
print('Sum of "Burnt area" polygons in square meters')

To verify the accuracy of my result, I researched the total area burnt by the Dixie fire. According to Wikipedia, that area was 
3,898 square kilometers, or 3,898,000 square meters; fairly close to my estimate. The main extent of the damage occurred in Plumas County, where several towns including Greenville, Canyondam, and Warner Valley were destroyed or damaged.